In [13]:
import joblib
import cv2
import kagglehub
import numpy as np
import os

from collections import defaultdict
from sklearn.model_selection import train_test_split
from src.image_classification.features import (
    AmountOfYellow,
    AmountOfSilver,
    AmountOfParallelLines,
    AmountOfCylinders,
    AmountOfReflections,
    AmountOfTransparency,
    AmountOfTextureSmoothness,
    AmountOfTextureShininess,
    AmountOfSurfaceAnisotropy,
    AmountOfAspectRatio,
    AmountOfWhiteness,
    AmountOfLineCurvature,
)

In [14]:
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
classes = os.listdir(os.path.join(path, "garbage-dataset"))


def get_features(img):
    return (AmountOfYellow()(img),
            AmountOfSilver()(img),
            AmountOfParallelLines()(img),
            AmountOfCylinders()(img),
            AmountOfReflections()(img),
            AmountOfTransparency()(img),
            AmountOfTextureSmoothness()(img),
            AmountOfTextureShininess()(img),
            AmountOfSurfaceAnisotropy()(img),
            AmountOfAspectRatio()(img),
            AmountOfWhiteness()(img),
            AmountOfLineCurvature()(img),)

test_images = [] 
features = []
labels = []
for class_name in classes:
    class_path = os.path.join(path + "/garbage-dataset/", class_name)
    image_names = os.listdir(class_path)

    # Split the images into training and testing sets (80% train, 20% test)
    train_images, test_images_class = train_test_split(image_names, test_size=0.2, random_state=42)

    # Process training images
    for image_name in train_images:
        image_path = os.path.join(class_path, image_name)
        image = cv2.imread(image_path)

        if image is not None:
            features_list = get_features(image)
            features.append(features_list)
            labels.append(class_name)

    # Collect test images for manual testing
    for image_name in test_images_class:
        test_images.append(os.path.join(class_path, image_name))

classifier = joblib.load('random_forest_classifier.pkl')

def classify_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        features_list = get_features(image)
        feature = np.array([features_list])  
        prediction = classifier.predict(feature)
        return prediction[0]
    else:
        return None

NameError: name 'image_names' is not defined

In [12]:
# Classifier testing (runs classifier for the remaining images)
def test_classify(test_images, target_class_name):
    predictions = defaultdict(int)
    for test_image in test_images:
        if target_class_name in test_image:  
            predicted_class = classify_image(test_image)  
            predictions[predicted_class] += 1 
    return dict(predictions)  

for class_name in classes:
    results = test_classify(test_images, class_name)
    print("Predicted class counts for test images containing " + class_name + ':')
    for class_name, count in results.items():
        print(f"Class: {class_name}, Count: {count}")

NameError: name 'test_images' is not defined